<a href="https://colab.research.google.com/github/Cynric1/smarthusky1/blob/main/nested_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# It is the work in term 1, including defining a tree, calculating the nest distance between trees and generating random trees.
class TreeNode:
    def __init__(self, layer, ancestor=None, transition_prob=None, value=None):
        self.layer = layer
        self.ancestor = ancestor
        self.transition_prob = transition_prob
        self.value = value
        self.descendants = []

    def add_descendant(self, node):
        self.descendants.append(node)

    def __repr__(self):
        ancestor_value = self.ancestor.value if self.ancestor else None
        return (f"TreeNode(layer={self.layer}, value={self.value}, "
                f"ancestor_value={ancestor_value}, "
                f"transition_prob={self.transition_prob})")

class Tree:
    def __init__(self):
        self.nodes = []

    def add_node(self, node):
        self.nodes.append(node)

    def get_nodes_at_layer(self, layer):
        return [node for node in self.nodes if node.layer == layer]

    def __repr__(self):
        return f"Tree with {len(self.nodes)} nodes:\n" + "\n".join([str(node) for node in self.nodes])

def compute_wasserstein_distance_with_cost(cost_matrix, p, max_iter=1000, tol=1e-12):
    m, n = cost_matrix.shape
    cost_matrix_p = cost_matrix ** p
    K = np.exp(-cost_matrix_p)
    u = np.ones(m)
    v = np.ones(n)
    p = np.ones(m) / m
    q = np.ones(n) / n
    max_iter = int(max_iter)

    for iteration in range(max_iter):
        u_prev, v_prev = u.copy(), v.copy()
        u = p / (K @ v)
        v = q / (K.T @ u)
        if np.linalg.norm(u - u_prev) < tol and np.linalg.norm(v - v_prev) < tol:
            break

    P = np.diag(u) @ K @ np.diag(v)
    distance = np.sum(P * cost_matrix_p)
    return distance, P

def calculate_local_distance(node_a, node_b, p):
    """Calculate the local distance between two nodes using canonical distance to the power of p."""
    return abs(node_a.value - node_b.value) ** p

def generate_random_tree(num_layers=10, max_children=2):
    tree = Tree()
    root = TreeNode(layer=0, value=random.uniform(0.99, 1.01)) # Here, I try to mimic the real daily return, and assume daily increases and decrease <1%
    tree.add_node(root)
    nodes_by_layer = {0: [root]}

    for layer in range(1, num_layers + 1):
        nodes_by_layer[layer] = []
        for ancestor in nodes_by_layer[layer - 1]:
            num_children = random.randint(1, max_children)
            # Normalize transition probabilities
            transition_probs = [random.uniform(0, 1) for _ in range(num_children)]
            total_prob = sum(transition_probs)
            transition_probs = [p/total_prob for p in transition_probs]

            for i in range(num_children):
                child_value = random.uniform(0.99, 1.01) # Similarly as before for each node
                transition_prob = transition_probs[i]
                child = TreeNode(layer=layer, ancestor=ancestor, transition_prob=transition_prob, value=child_value)
                ancestor.add_descendant(child)
                tree.add_node(child)
                nodes_by_layer[layer].append(child)

    return tree

def compute_nested_wasserstein_distance(tree_a, tree_b, p=1, max_iter=1000, tol=1e-12):
    """
    Compute the nested Wasserstein distance between two trees, storing coupling matrices for node pairs.
    Args:
        tree_a (Tree): The first tree.
        tree_b (Tree): The second tree.
        p (int): The power to which the canonical distance is raised.
        max_iter (int): Maximum iterations for Sinkhorn's algorithm.
        tol (float): Convergence tolerance for Sinkhorn's algorithm.
    Returns:
        total_distance (float): The nested Wasserstein distance between the two trees.
        node_distances (dict): Wasserstein distances for each node pair.
        node_coupling_matrices (dict): Coupling matrices for each node pair.
    """
    node_distances = {}
    node_coupling_matrices = {}

    max_layer_a = max([node.layer for node in tree_a.nodes]) if tree_a.nodes else 0
    max_layer_b = max([node.layer for node in tree_b.nodes]) if tree_b.nodes else 0
    layer = max(max_layer_a, max_layer_b)

    # Leaf layer
    leaf_nodes_a = tree_a.get_nodes_at_layer(layer)
    leaf_nodes_b = tree_b.get_nodes_at_layer(layer)

    if not leaf_nodes_a or not leaf_nodes_b:
        return float('inf'), node_distances, node_coupling_matrices

    cost_matrix = np.zeros((len(leaf_nodes_a), len(leaf_nodes_b)))
    for i, node_a in enumerate(leaf_nodes_a):
        for j, node_b in enumerate(leaf_nodes_b):
            cost_matrix[i, j] = calculate_local_distance(node_a, node_b, p)

    distance, coupling = compute_wasserstein_distance_with_cost(cost_matrix, p, max_iter, tol)

    # Store distance and coupling for each pair of leaf nodes
    for i in range(len(leaf_nodes_a)):
        for j in range(len(leaf_nodes_b)):
            node_distances[(layer, i, j)] = distance
            node_coupling_matrices[(layer, i, j)] = coupling

    # Backwards
    for layer in range(layer - 1, -1, -1):
        nodes_a = tree_a.get_nodes_at_layer(layer)
        nodes_b = tree_b.get_nodes_at_layer(layer)

        for i, node_a in enumerate(nodes_a):
            for j, node_b in enumerate(nodes_b):
                descendants_a = node_a.descendants
                descendants_b = node_b.descendants

                if not descendants_a or not descendants_b:
                    node_distances[(layer, i, j)] = calculate_local_distance(node_a, node_b, p)
                    node_coupling_matrices[(layer, i, j)] = np.array([[1]])
                    continue

                descendant_cost_matrix = np.zeros((len(descendants_a), len(descendants_b)))

                for k, descendant_a in enumerate(descendants_a):
                    descendant_layer_a = descendant_a.layer
                    descendant_idx_a = tree_a.get_nodes_at_layer(descendant_layer_a).index(descendant_a)

                    for l, descendant_b in enumerate(descendants_b):
                        descendant_layer_b = descendant_b.layer
                        descendant_idx_b = tree_b.get_nodes_at_layer(descendant_layer_b).index(descendant_b)

                        # Distance between descendants
                        key = (descendant_layer_a, descendant_idx_a, descendant_idx_b)
                        if key in node_distances:
                            descendant_cost_matrix[k, l] = node_distances[key]
                        else:
                            # If distance not computed, use local distance as fallback
                            descendant_cost_matrix[k, l] = calculate_local_distance(descendant_a, descendant_b, p)

                # Compute Wasserstein distance and coupling matrix for the current node pair
                descendant_distance, descendant_coupling_matrix = compute_wasserstein_distance_with_cost(
                    descendant_cost_matrix, p, max_iter, tol)

                node_distances[(layer, i, j)] = descendant_distance + calculate_local_distance(node_a, node_b, p)
                node_coupling_matrices[(layer, i, j)] = descendant_coupling_matrix

    if (0, 0, 0) in node_distances:
        root_distance = node_distances[(0, 0, 0)]
    else:
        # Fallback if root distance not calculated
        root_distance = float('inf')

    return root_distance, node_distances, node_coupling_matrices

def sample_path_from_tree(tree, num_layers=None):
    """Sample a path from a tree based on transition probabilities."""
    if not tree.nodes:
        return []

    # Find the root node
    root = None
    for node in tree.nodes:
        if node.layer == 0:
            root = node
            break

    if not root:
        return []

    # Initialize the path with the root node
    path = [root]
    current_node = root

    # Sample nodes for each subsequent layer
    for layer in range(1, num_layers):
        descendants = current_node.descendants

        if not descendants:
            break

        # Get transition probabilities and sample next node
        probs = [node.transition_prob for node in descendants]
        next_node = random.choices(descendants, weights=probs, k=1)[0]
        path.append(next_node)
        current_node = next_node

    return path

def path_to_values_vector(path):
    """Convert a path of TreeNodes to values."""
    expected_length = 10 # 10 just in this example
    values = [node.value for node in path]
    return np.array(values)

def path_to_binary_label(path):
    """
    Convert a path to binary labels (1 or -1) based on node values.
    If node value > 1, then 1 (price increase), otherwise -1 (price decrease or flat).
    """
    expected_length = 10
    labels = [1 if node.value > 1 else -1 for node in path]
    return np.array(labels)

def create_neural_network(input_size=10):

    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_size,)),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(input_size, activation='tanh')
    ])

    return model


def calculate_accuracy(predictions, ground_truth):
    """
    Calculate accuracy between binary predictions and ground truth.

    Args:
        predictions (numpy.ndarray): Predicted values.
        ground_truth (numpy.ndarray): Ground truth values.

    Returns:
        float: Accuracy as a percentage.
    """
    # Convert predictions to binary classes
    binary_predictions = np.where(predictions > 0, 1, -1)

    # Calculate accuracy
    correct = np.sum(binary_predictions == ground_truth)
    total = len(ground_truth)

    return correct / total


def generate_and_filter_trees(num_nu_trees=200, num_layers=10, max_children=2, distance_threshold=0.05):
    """
    Generate a reference tree (mu) and multiple comparison trees (nu_1 to nu_n),
    then filter based on distance threshold.

    Args:
        num_nu_trees (int): Number of nu trees to generate.
        num_layers (int): Number of layers in each tree.
        max_children (int): Maximum number of children per node.
        distance_threshold (float): Maximum distance for filtering.

    Returns:
        tuple: (mu_tree, mu_path, filtered_nu_trees, filtered_nu_paths, filtered_distances)
    """
    # Generate reference tree (mu)
    mu_tree = generate_random_tree(num_layers, max_children)

    # Sample a single path from mu
    mu_path = sample_path_from_tree(mu_tree, num_layers)

    # Generate comparison trees (nu_1 to nu_n)
    nu_trees = []
    nu_paths = []
    distances = []

    print(f"Generating and filtering {num_nu_trees} nu trees...")

    # Generate nu trees and compute distances
    for i in range(num_nu_trees):
        nu_tree = generate_random_tree(num_layers, max_children)
        nu_path = sample_path_from_tree(nu_tree, num_layers)

        # Compute distance between mu and nu
        distance, _, _ = compute_nested_wasserstein_distance(mu_tree, nu_tree)

        if distance < distance_threshold:
            nu_trees.append(nu_tree)
            nu_paths.append(nu_path)
            distances.append(distance)

        if (i+1) % 20 == 0:
            print(f"Processed {i+1}/{num_nu_trees} trees, found {len(nu_trees)} within threshold")

    # Convert distances to numpy array
    filtered_distances = np.array(distances)

    print(f"Found {len(nu_trees)} trees with distance < {distance_threshold}")

    return mu_tree, mu_path, nu_trees, nu_paths, filtered_distances

def distributionally_robust_loss(batch_weights=None):
    """
    Create a custom loss function that emphasizes the worst-case examples.

    Args:
        batch_weights: Optional weights to apply to each example in the batch.

    Returns:
        A loss function that can be used by the Keras model.
    """
    def dro_loss(y_true, y_pred):
        # Calculate squared error for each sample
        squared_errors = tf.reduce_mean(tf.square(y_pred - y_true), axis=1)
        return tf.reduce_max(squared_errors)

    return dro_loss

def train_neural_network_dro(mu_path, nu_paths, epochs=50, batch_size=16, patience=10):
    """Train a neural network using the DRO approach."""
    mu_values = path_to_values_vector(mu_path)
    mu_labels = path_to_binary_label(mu_path)

    nu_values = np.array([path_to_values_vector(path) for path in nu_paths])
    nu_labels = np.array([path_to_binary_label(path) for path in nu_paths])

    model = create_neural_network(input_size=10)

    # Training history to see how max_error changes
    max_error_history = []

    # Early stopping
    best_max_error = float('inf')
    best_model_weights = None
    patience_counter = 0

    print(f"Training neural network with {len(nu_paths)} nu paths using DRO approach...")

    # Combined dataset for evaluation
    all_values = np.vstack([nu_values])
    all_labels = np.vstack([nu_labels])

    for epoch in range(epochs):
        # identify the worst-case distribution (nu path with max error)
        all_error_rates = []

        for i, (values, labels) in enumerate(zip(nu_values, nu_labels)):
            # Predict
            predictions = model.predict(np.array([values]), verbose=0)[0]

            # Convert to binary predictions (1 if > 1, else -1)
            binary_predictions = np.where(predictions > 1, 1, -1)

            # Calculate error rate
            error_rate = 1.0 - np.mean(binary_predictions == labels)
            all_error_rates.append(error_rate)

        # Find the worst-case path (highest error)
        if all_error_rates:
            max_error_idx = np.argmax(all_error_rates)
            max_error = all_error_rates[max_error_idx]
            max_error_history.append(max_error)

        # Early stopping check
        if max_error < best_max_error:
            best_max_error = max_error
            best_model_weights = model.get_weights()
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}. No improvement for {patience} epochs.")
            # Restore best weights
            model.set_weights(best_model_weights)
            break

        # emphasize the worst-case paths
        sample_weights = np.ones(len(nu_paths))
        sample_weights[max_error_idx] = 5.0  # Give 5x weight to the worst-case path

        # Create a custom loss that focuses on the worst-case
        custom_loss = distributionally_robust_loss()
        model.compile(optimizer=Adam(learning_rate=0.001), loss=custom_loss)

        # Train with custom weights
        model.fit(
            all_values,
            all_labels,
            sample_weight=sample_weights,
            epochs=1,
            verbose=0,
            batch_size=batch_size
        )

        if (epoch + 1) % 5 == 0 or epoch == 0:
            print(f"Epoch {epoch+1}/{epochs}: Worst-case error rate = {max_error:.4f} (path {max_error_idx+1}), Best = {best_max_error:.4f}")

    # Make sure we're using the best model weights
    if best_model_weights is not None:
        model.set_weights(best_model_weights)

    return model

def main_algorithm1():
    # Parameters
    num_nu_trees = 200  # Number of trees
    num_layers = 10     # Number of layers in each tree
    max_children = 2    # Maximum number of children per node
    distance_threshold = 0.08  # Maximum distance for filtering
    early_stopping_patience = 10  # Number of epochs with no improvement before stopping

    random.seed(42)
    np.random.seed(42)
    tf.random.set_seed(42)

    # Generate reference tree (mu) and sample a path
    print("Generating mu tree...")
    mu_tree = generate_random_tree(num_layers, max_children)
    mu_path = sample_path_from_tree(mu_tree, num_layers)

    # Generate and filter nu trees
    print("\nGenerating and filtering nu trees for DRO algorithm...")
    _, _, nu_trees, nu_paths, distances = generate_and_filter_trees(
        num_nu_trees=num_nu_trees,
        num_layers=num_layers,
        max_children=max_children,
        distance_threshold=distance_threshold
    )

    if len(nu_trees) < 10:
        print("Not enough trees found within threshold.")
        distance_threshold *= 2
        _, _, nu_trees, nu_paths, distances = generate_and_filter_trees(
            num_nu_trees=num_nu_trees,
            num_layers=num_layers,
            max_children=max_children,
            distance_threshold=distance_threshold
        )

    # Split data into training and testing sets
    n_total = len(nu_trees)
    n_test = max(1, int(n_total * 0.1))  # 10% for testing
    n_train = n_total - n_test  # Rest for training

    # Create indices for splitting
    indices = np.arange(n_total)
    np.random.shuffle(indices)

    test_indices = indices[:n_test]
    train_indices = indices[n_test:]

    # Extract datasets
    train_nu_paths = [nu_paths[i] for i in train_indices]
    test_nu_paths = [nu_paths[i] for i in test_indices]

    train_distances = distances[train_indices]
    test_distances = distances[test_indices]

    print(f"\nData split:")
    print(f"Training: {len(train_nu_paths)} paths")
    print(f"Testing: {len(test_nu_paths)} paths")

    # ===== ALGORITHM 1: Train with DRO approach =====
    print("\n===== ALGORITHM 1: Distributionally Robust Optimization =====")
    dro_model = train_neural_network_dro(
        mu_path, train_nu_paths,
        epochs=50, batch_size=16, patience=early_stopping_patience
    )

    # Evaluate on test set
    print("\n===== EVALUATING DRO MODEL ON TEST SET =====")
    test_values = np.array([path_to_values_vector(path) for path in test_nu_paths])
    test_labels = np.array([path_to_binary_label(path) for path in test_nu_paths])

    # Get predictions
    test_predictions = dro_model.predict(test_values, verbose=0)
    test_binary_predictions = np.where(test_predictions > 1, 1, -1)

    # Calculate accuracy and error rates
    accuracy_rates = []
    error_rates = []

    for i in range(len(test_nu_paths)):
        accuracy = np.mean(test_binary_predictions[i] == test_labels[i])
        error = 1.0 - accuracy

        accuracy_rates.append(accuracy)
        error_rates.append(error)

        print(f"Test path {i+1}: Accuracy = {accuracy:.4f}, Error = {error:.4f}, Distance = {test_distances[i]:.4f}")

    mean_accuracy = np.mean(accuracy_rates)
    mean_error = np.mean(error_rates)
    max_error = np.max(error_rates)
    max_error_idx = np.argmax(error_rates)

    print(f"\nTest set results:")
    print(f"Mean accuracy: {mean_accuracy:.4f}")
    print(f"Mean error rate: {mean_error:.4f}")
    print(f"Maximum error rate: {max_error:.4f} (path {max_error_idx+1})")

    # Plot error rates
    plt.figure(figsize=(10, 6))
    plt.bar(range(1, len(error_rates) + 1), error_rates)
    plt.xlabel('Test Path Index')
    plt.ylabel('Error Rate')
    plt.title('DRO Model Error Rates on Test Set')
    plt.grid(True, alpha=0.3)
    plt.savefig('dro_test_error_rates.png')
    plt.close()

    # Plot relationship between distance and error
    plt.figure(figsize=(10, 6))
    plt.scatter(test_distances, error_rates, alpha=0.7)
    plt.xlabel('Nested Wasserstein Distance')
    plt.ylabel('Error Rate')
    plt.title('Relationship Between Distance and Error Rate (DRO Model)')
    plt.grid(True, alpha=0.3)
    plt.savefig('dro_distance_vs_error.png')
    plt.close()

    return {
        'mu_path': mu_path,
        'train_paths': train_nu_paths,
        'test_paths': test_nu_paths,
        'test_values': test_values,
        'test_labels': test_labels,
        'test_distances': test_distances,
        'dro_model': dro_model,
        'dro_predictions': test_predictions,
        'dro_binary_predictions': test_binary_predictions,
        'accuracy_rates': accuracy_rates,
        'error_rates': error_rates,
        'mean_accuracy': mean_accuracy,
        'mean_error': mean_error,
        'max_error': max_error
    }

if __name__ == "__main__":
    results = main_algorithm1()

Generating mu tree...

Generating and filtering nu trees for DRO algorithm...
Generating and filtering 200 nu trees...
Processed 20/200 trees, found 13 within threshold
Processed 40/200 trees, found 29 within threshold
Processed 60/200 trees, found 48 within threshold
Processed 80/200 trees, found 63 within threshold
Processed 100/200 trees, found 81 within threshold
Processed 120/200 trees, found 100 within threshold
Processed 140/200 trees, found 115 within threshold
Processed 160/200 trees, found 133 within threshold
Processed 180/200 trees, found 150 within threshold
Processed 200/200 trees, found 169 within threshold
Found 169 trees with distance < 0.08

Data split:
Training: 153 paths
Testing: 16 paths

===== ALGORITHM 1: Distributionally Robust Optimization =====
Training neural network with 153 nu paths using DRO approach...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50: Worst-case error rate = 0.8000 (path 34), Best = 0.8000
Epoch 5/50: Worst-case error rate = 0.8000 (path 34), Best = 0.8000
Epoch 10/50: Worst-case error rate = 0.8000 (path 34), Best = 0.8000
Early stopping at epoch 11. No improvement for 10 epochs.

===== EVALUATING DRO MODEL ON TEST SET =====
Test path 1: Accuracy = 0.7000, Error = 0.3000, Distance = 0.0725
Test path 2: Accuracy = 0.4000, Error = 0.6000, Distance = 0.0698
Test path 3: Accuracy = 0.5000, Error = 0.5000, Distance = 0.0656
Test path 4: Accuracy = 0.7000, Error = 0.3000, Distance = 0.0768
Test path 5: Accuracy = 0.5000, Error = 0.5000, Distance = 0.0724
Test path 6: Accuracy = 0.3000, Error = 0.7000, Distance = 0.0690
Test path 7: Accuracy = 0.6000, Error = 0.4000, Distance = 0.0770
Test path 8: Accuracy = 0.3000, Error = 0.7000, Distance = 0.0662
Test path 9: Accuracy = 0.8000, Error = 0.2000, Distance = 0.0800
Test path 10: Accuracy = 0.4000, Error = 0.6000, Distance = 0.0711
Test path 11: Accuracy = 0.800